In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")

In [5]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*********************100%***********************]  7384 of 7384 completed

59 Failed downloads:
- BLEUR: Period '1y' is invalid, must be one of ['1d', '5d']
- SVIIR: Period '1y' is invalid, must be one of ['1d', '5d']
- LBBBR: Period '1y' is invalid, must be one of ['1d', '5d']
- IGTAR: Period '1y' is invalid, must be one of ['1d', '5d']
- WTMAR: Period '1y' is invalid, must be one of ['1d', '5d']
- ONFOW: Period '1y' is invalid, must be one of ['1d', '5d']
- HSPOR: Period '1y' is invalid, must be one of ['1d', '5d']
- CSLMR: Period '1y' is invalid, must be one of ['1d', '5d']
- FLFVR: Period '1y' is invalid, must be one of ['1d', '5d']
- BRACR: Period '1y' is invalid, must be one of ['1d', '5d']
- STRCW: Period '1y' is invalid, must be one of ['1d', '5d']
- ADOCR: Period '1y' is invalid, must be one of ['1d', '5d']
- HHGCR: Period '1y' is invalid, must be one of ['1d', '5d']
- ATAKR: Period '1y' is invalid, must be one of ['1d', '5d']
- ALSAR: Period '1y' is invalid, must be one of [

In [6]:
df = get_info(data)
cop = df.copy()
df

,Close,52wk High,Percent from 52,Volume,doh,Volume$
A,122.68,160.26,23.45,"1,157,900.00",2022-12-13,"142,051,172.35"
AA,29.51,57.61,48.78,"4,926,600.00",2023-01-18,"145,383,967.13"
AAC,10.70,10.70,0.00,"5,000.00",2023-08-30,"53,500.00"
AACG,1.26,2.50,49.60,"6,200.00",2022-09-01,"7,812.00"
AACI,10.53,12.20,13.69,"8,100.00",2023-01-30,"85,293.00"
...,...,...,...,...,...,...
ZVSA,0.14,25.00,99.43,"489,500.00",2022-12-12,"69,509.00"
ZWS,29.57,30.87,4.21,"525,600.00",2023-07-26,"15,541,991.84"
ZYME,7.35,10.80,31.94,"277,800.00",2023-01-19,"2,041,829.97"
ZYNE,1.32,1.38,4.35,"499,700.00",2023-08-14,"659,604.03"


### Filtering out stocks that don't match 

In [7]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 10000
vold_thresh = 500000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=200)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(15)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
ENSG,101.77,102.26,0.48,"401,500.00",2023-04-25,"40,860,653.65"
FLME,10.39,10.47,0.76,"301,600.00",2023-03-03,"3,133,624.10"
AVPT,6.87,6.93,0.87,"921,000.00",2023-06-06,"6,327,269.89"
EFR,12.10,12.23,1.06,"59,500.00",2023-02-16,"719,950.02"
NEWR,85.07,86.00,1.08,"378,100.00",2023-05-17,"32,164,966.88"
ARGO,29.75,30.13,1.26,"431,200.00",2023-04-06,"12,828,200.00"
CHKP,134.17,135.93,1.29,"537,600.00",2023-04-12,"72,129,791.02"
ACN,323.25,327.93,1.43,"1,470,400.00",2023-06-16,"475,306,800.00"
MCAA,11.04,11.20,1.43,"99,200.00",2023-06-09,"1,095,168.00"
GHL,14.75,14.97,1.47,"128,400.00",2023-06-05,"1,893,900.00"


# Testing out (manually backtesting)

In [32]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [33]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,127.62,160.26,20.37,"1,333,326.33",2022-12-13,126.87,20.83,160.26,2022-12-13
AA,33.38,58.24,42.68,"3,132,267.33",2022-08-26,33.80,41.96,58.24,2022-08-26
AAC,10.56,10.60,0.38,"21,163.33",2023-07-18,10.65,0.05,10.65,2023-07-31
AACG,1.39,2.64,47.35,"2,835.00",2022-08-18,1.32,50.00,2.64,2022-08-18
AACI,10.51,12.20,13.85,"8,276.50",2023-01-30,NaN,NaN,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.20,25.00,99.21,"1,000,905.00",2022-12-12,0.17,99.33,25.00,2022-12-12
ZWS,27.33,31.77,13.98,"590,255.00",2022-08-18,30.21,4.91,31.77,2022-08-18
ZYME,7.87,10.80,27.13,"923,709.67",2023-01-19,7.16,33.66,10.80,2023-01-19
ZYNE,0.36,1.39,74.10,"96,968.00",2022-08-08,0.35,74.97,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [6]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [7]:
def parse_df(df, old_high=2, high_by="High"):
    # old_high- How many days ago it broke high (x or less)
    # high_by- by which OHLC to determine d
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})

    return new_df

### Filtering and sorting DataFrame

In [8]:
def filter_sort_df(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of high"] != df["Date of high"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [9]:
full_df = parse_df(data, 2)
cop = full_df.copy()

df = filter_sort_df(cop)
df

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$
TNET,110.93,112.13,1.07,"890,100.00",2023-08-31,2023-06-09,"98,738,793.27"
CAE,24.11,24.42,1.27,"185,100.00",2023-08-31,2023-04-12,"4,462,761.11"
AVPT,6.94,7.05,1.56,"1,870,800.00",2023-08-31,2023-06-06,"12,983,352.11"
NTRA,58.73,59.89,1.94,"1,102,300.00",2023-08-30,2023-03-09,"64,738,078.50"
ENSG,100.22,104.04,3.67,"541,500.00",2023-08-31,2023-04-25,"54,269,130.66"
USAC,21.22,22.09,3.94,"519,800.00",2023-08-31,2023-03-03,"11,030,155.64"
GFGD,8.23,11.00,25.18,"71,200.00",2023-08-30,2023-04-12,"585,975.97"
CHSN,2.56,4.28,40.12,"743,900.00",2023-08-30,2023-03-30,"1,904,383.96"


In [ ]:
# add_to_tv(df)

In [14]:
cop.loc["URI"]

Close                            449.41
52wk High                        492.22
% to 52wk                          8.70
Volume                        24,350.00
Date of high        2023-08-07 00:00:00
Old date of high    2023-08-07 00:00:00
Volume$                   10,943,133.59
Name: URI, dtype: object

# Finding low 52

In [25]:
def parse_df_low(df, old_low=2, low_by="Low"):
    # old_low- How many days ago it broke low (x or less)
    # low_by- by which OHLC to determine d
    low = df['Low'].min()  # 52 wk low
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((low - close) / low) * 100  # Percent change to 52wk low
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['Low'][:-old_low].idxmin()  # 52wk low before 2 days
    doh = df['Low'].idxmin()  # 52wk low today
    new_df = pd.DataFrame({'Close': close, '52wk Low': low, "% to 52wk": pct_to_52, "Volume": volume, "Date of low": doh, "Old date of low": old_doh, "Volume$": volume * close})

    return new_df

def filter_sort_df_low(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a low volume but due to low price they have a low volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the low between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of low"] < time_thresh1) & (df["Old date of low"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of low"] != df["Date of low"]]
    
    # return df.sort_values(by='% to 52wk', ascending=True)
    return df.sort_values(by='Date of low', ascending=True)

In [26]:
full_df = parse_df_low(data, 12)
cop = full_df.copy()

df = filter_sort_df_low(cop)
df

,Close,52wk Low,% to 52wk,Volume,Date of low,Old date of low,Volume$
ADTN,8.17,7.00,-16.68,"309,340.00",2023-08-07,2023-05-09,"2,526,534.31"
BNTX,123.35,95.50,-29.16,"1,124,301.00",2023-08-07,2023-05-11,"138,682,526.63"
CRON,1.78,1.64,-8.54,"957,938.00",2023-08-08,2023-06-06,"1,705,129.61"
AGEN,1.47,1.18,-24.58,"1,743,553.00",2023-08-08,2023-04-12,"2,563,022.96"
G,36.17,35.31,-2.44,"662,809.00",2023-08-08,2023-05-11,"23,973,800.32"
...,...,...,...,...,...,...,...
OCGN,0.40,0.40,-1.44,"7,020,635.00",2023-08-22,2023-05-26,"2,827,209.76"
CUTR,11.06,10.54,-4.93,"258,079.00",2023-08-22,2023-05-12,"2,854,353.85"
ONTF,6.80,6.60,-3.11,"240,340.00",2023-08-22,2023-05-22,"1,635,513.66"
CTVA,49.28,49.24,-0.10,"1,799,940.00",2023-08-22,2023-06-01,"88,710,042.63"
